In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import glob
import os
import re
# Absolute Data directory of ARPA data
dd_arpa = "c:\\Users\\m1865\\Desktop\\GIS Thesis Python\\ARPA"
# Set current working directory
cwd = os.getcwd()
cwd

In [ ]:
# Create an empty file list where all the txt files downloaded from ARPA will be imported and saved
myFile_NO2 = []
myFile_O3 = []
myFile_PM25 = []
myFile_PM10 = []
# Filter all the file names of the data from the data folder
for filename in os.listdir(dd_arpa):
    match = re.match(r'(.*)_2021(.*).txt', filename)
    # print('Check Match')
    if match:
        # print('Matched!')
        if match.group(1) == "NO2":
            myFile_NO2.append(filename)
        elif match.group(1) == "O3":
            myFile_O3.append(filename)
        elif match.group(1) == "PM25": 
            myFile_PM25.append(filename)
        elif match.group(1) == "PM10":
            myFile_PM10.append(filename)
# myFile_ALL

In [ ]:
# Read each txt file from the file list as dataframe and save into a new dataframe list
myList_NO2 = []
myList_O3 = []
myList_PM10 = []
myList_PM25 = []
for filename in myFile_NO2:
    df = pd.read_csv(dd_arpa + "\\" + filename, sep=";")
    myList_NO2.append(df)
for filename in myFile_O3:
    df = pd.read_csv(dd_arpa + "\\" + filename, sep=";")
    myList_O3.append(df)
for filename in myFile_PM10:
    df = pd.read_csv(dd_arpa + "\\" + filename, sep=";")
    myList_PM10.append(df)
for filename in myFile_PM25:
    df = pd.read_csv(dd_arpa + "\\" + filename, sep=";")
    myList_PM25.append(df)

In [ ]:
# Check if it is correct
# myList_NO2[0]

In [ ]:
# Concatenate the dataframe in the list into one dataframe
arpa_NO2 = pd.concat(myList_NO2, axis=0, ignore_index=True)
arpa_O3 = pd.concat(myList_O3, axis=0, ignore_index=True)
arpa_PM10 = pd.concat(myList_PM10, axis=0, ignore_index=True)
arpa_PM25 = pd.concat(myList_PM25, axis=0, ignore_index=True)
# arpa_NO2

In [ ]:
myList_ALL = [arpa_NO2, arpa_O3, arpa_PM10, arpa_PM25]
arpa_ALL = pd.concat(myList_ALL, axis=0, ignore_index=True)

In [ ]:
# Convert to datetime, and assign two new columns for month and day
arpa_ALL['Data'] = pd.to_datetime(arpa_ALL['Data'],format='%d/%m/%Y')
arpa_ALL['Month'] = arpa_ALL['Data'].dt.month
arpa_ALL['Day'] = arpa_ALL['Data'].dt.day

In [ ]:
# Drop useless columns
arpa_ALL = arpa_ALL.drop(columns=['Data','Cons'])
# Reorder columns
arpa_ALL = arpa_ALL[['Inquinante', 'Valore(ug/m3)','Month','Day','Cella_X','Cella_Y','Ora']]
# Rename for better usage
arpa_ALL.rename(columns={'Valore(ug/m3)': 'Valore'}, inplace=True)

In [ ]:
arpa_ALL

In [ ]:
df_refer = pd.read_csv(dd_arpa + "\\" + "Dominio_1km.csv", sep=";")
# Rename the common two columns in order to merge with df_data
df_refer.rename(columns={'CELLA_Y':'Cella_Y','CELLA_X':'Cella_X'},inplace=True)
df_refer

In [ ]:
df_arpa = pd.merge(arpa_ALL,df_refer,left_on=['Cella_X','Cella_Y'],right_on=['Cella_X','Cella_Y'])
df_arpa

In [ ]:
df_arpa_mean = df_arpa.groupby(["Inquinante","Month","ID_CELLA"],as_index=False).mean()
df_arpa_mean = df_arpa_mean.drop(columns=["Day","Ora"])

In [ ]:
df_arpa_mean

In [ ]:
df_arpa_mean.to_csv(cwd + "\\arpa_2021_all.csv",index=False, chunksize=100000)

In [ ]:
df_arpa_mean[df_arpa_mean['Inquinante']=='NO2'].to_csv(cwd + "\\ARPA NO2.csv", index=False)
df_arpa_mean[df_arpa_mean['Inquinante']=='O3'].to_csv(cwd + "\\ARPA O3.csv", index=False)
df_arpa_mean[df_arpa_mean['Inquinante']=='PM10'].to_csv(cwd + "\\ARPA PM10.csv", index=False)
df_arpa_mean[df_arpa_mean['Inquinante']=='PM25'].to_csv(cwd + "\\ARPA PM25.csv", index=False)